#### Testing combinations(x,y) with starts and stops as indices

This is intended to be used with oamap version ( I know we don't intend to finally use it, but let's go with it for now).

##### Idea for calculation

Using the kernel structure used in test_combinations, we can  generate combinations arrays for every event. 

For that, each event is assigned a block ( i.e. each block in the grid serves as an event), and the combinations is calculated for each block using the `starts[block_index]` and event length.
This is appended to `left` and `right` arrays. 

In [46]:
import pycuda.autoinit
import pycuda
import pycuda.driver as cuda
import pycuda.gpuarray as gpuarray
from pycuda.compiler import *
import numpy

In [47]:
# The generation step. Let's form a random integer array, from which we will form starts 
# and stops arrays
base_len = 32
base_arr1 = numpy.random.randint(6, size=base_len)
base_arr2 = numpy.random.randint(4, size=base_len)

In [48]:
cumul_arr1 = numpy.zeros(base_len+1)
cumul_arr1[0] = 0
cumul_arr1[1:] = numpy.cumsum(base_arr1)

cumul_arr2 = numpy.zeros(base_len+1)
cumul_arr2[0] = 0
cumul_arr2[1:] = numpy.cumsum(base_arr2)

In [49]:
start1 = cumul_arr1[:-2]
stop1 = cumul_arr1[1:-1]
start2 = cumul_arr2[:-2]
stop2 = cumul_arr2[1:-1]

In [50]:
lengths1 = stop1-start1
lengths2 = stop2-start2
pairs_lengths = numpy.zeros(len(lengths1)+1, dtype=numpy.int32)
pairs_lengths[1:] = numpy.cumsum(lengths1*lengths2)
lengths_arr = numpy.array([base_len-1]).astype(numpy.int32)
lengths1 = lengths1.astype(numpy.int32)
start1 = start1.astype(numpy.int32)
lengths2 = lengths2.astype(numpy.int32)
start2 = start2.astype(numpy.int32)

In [52]:
# Now let's form the cuda function

mod = SourceModule('''
__global__ void comb_events(int* left,int* right,int* start1,int* start2,int* length,int* lengths1,int* lengths2,int* pairs_lengths)
{
    int i = blockIdx.x*blockDim.x + threadIdx.x;
    int j =  blockIdx.y*blockDim.y+threadIdx.y;
    int k =  blockIdx.z*blockDim.z+threadIdx.z;
    if (i <length[0])
    {
    if (j< lengths1[i] && k<lengths2[i])
        {
            left[pairs_lengths[i] + j*lengths2[i] + k] = j + start1[i];
            right[pairs_lengths[i] + j*lengths2[i] + k] = k + start2[i];
        }
    }
}
''')

C:\Users\Jaydeep\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: The CUDA compiler succeeded, but said the following:
kernel.cu



In [53]:
func = mod.get_function("comb_events")

In [54]:
left = numpy.empty(pairs_lengths[-1]).astype(numpy.int32)
right = numpy.empty(pairs_lengths[-1]).astype(numpy.int32)

In [55]:
func(cuda.InOut(left), cuda.InOut(right), cuda.In(start1),cuda.In(start2), cuda.In(lengths_arr),
    cuda.In(lengths1),cuda.In(lengths2), cuda.In(pairs_lengths), block=(1, 8, 8), grid=(base_len-1, 1))

In [56]:
left

array([ 6,  6,  6,  7,  7,  7,  8,  8,  8,  9,  9,  9, 10, 10, 10, 11, 11,
       11, 12, 12, 12, 13, 13, 13, 21, 21, 21, 32, 32, 32, 33, 33, 33, 37,
       37, 37, 38, 38, 38, 39, 39, 39, 40, 40, 40, 41, 41, 41, 42, 42, 42,
       43, 43, 43, 44, 44, 44, 45, 45, 45, 46, 46, 46, 52, 52, 53, 53, 54,
       54, 55, 55, 56, 56, 57, 58])

In [57]:
#Let's reshape it
for i in range(6):
    print("Event: {} \n Left:{} \n Right:{} \n".format(i, left[pairs_lengths[i]:pairs_lengths[i+1]], right[pairs_lengths[i]:pairs_lengths[i+1]]))

Event: 0 
 Left:[] 
 Right:[] 

Event: 1 
 Left:[] 
 Right:[] 

Event: 2 
 Left:[] 
 Right:[] 

Event: 3 
 Left:[6 6 6 7 7 7 8 8 8] 
 Right:[0 1 2 0 1 2 0 1 2] 

Event: 4 
 Left:[9 9 9] 
 Right:[3 4 5] 

Event: 5 
 Left:[10 10 10 11 11 11 12 12 12 13 13 13] 
 Right:[6 7 8 6 7 8 6 7 8 6 7 8] 

